In [29]:
import pandas as pd
import talib
import datetime as dt
import hmac
import hashlib
import json
import time
import requests

In [30]:
API_Key = ""
API_Secret = ""
secret_bytes = bytes(API_Secret, encoding='utf-8')
timeStamp = int(round(time.time() * 1000))

def data_downloader(pair,interval,limit = 50,exchange = "I"):
    url = f"https://public.coindcx.com/market_data/candles?pair={exchange}-{pair}&interval={interval}&limit={limit}" 
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    df['date_time'] = df['time'].apply(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(int(str(x)[:-3]))))
    df['date_time'] = pd.to_datetime(df['date_time'])
    return df

def place_order(side:str,pair:str,price:float,quantity:float):
    body = {
        "side": side,"order_type": "limit_order",
        "market": pair,"price_per_unit": price,
        "total_quantity": quantity,"timestamp": timeStamp,
        "client_order_id": "2022.02.14-btcinr_1"}
    json_body = json.dumps(body, separators = (',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    url = "https://api.coindcx.com/exchange/v1/orders/create"
    headers = {'Content-Type': 'application/json','X-AUTH-APIKEY': API_Key,'X-AUTH-SIGNATURE': signature}
    response = requests.post(url, data = json_body, headers = headers)
    data = response.json()
    print(data)

def get_acc_bal():
    body = {
        "timestamp": timeStamp
    }
    
    json_body = json.dumps(body, separators = (',', ':'))
    
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    url = "https://api.coindcx.com/exchange/v1/users/balances"
    
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': API_Key,
        'X-AUTH-SIGNATURE': signature
    }
    
    response = requests.post(url, data = json_body, headers = headers)
    data = response.json();
    print(data)


In [31]:
df = data_downloader("BTC_INR", "1m", "50","I")
df['rsi'] = talib.RSI(df["close"],timeperiod=15)
df[["date_time","open","high","low","volume","close","rsi"]]

,date_time,open,high,low,volume,close,rsi
0,2024-04-30 13:20:00,5825999.70,5825999.70,5825999.70,0.000000,5825999.70,NaN
1,2024-04-30 13:19:00,5826999.80,5826999.80,5749501.32,0.001040,5825999.70,NaN
2,2024-04-30 13:18:00,5827999.80,5828999.99,5748301.18,0.000396,5826999.80,NaN
3,2024-04-30 13:17:00,5827999.99,5829999.99,5826999.78,0.000374,5827999.80,NaN
4,2024-04-30 13:16:00,5831898.78,5834899.00,5827999.98,0.000323,5827999.99,NaN
5,2024-04-30 13:15:00,5834898.99,5834898.99,5831898.78,0.000119,5831898.78,NaN
6,2024-04-30 13:14:00,5742200.50,5834898.99,5742200.50,0.000202,5834898.99,NaN
7,2024-04-30 13:13:00,5832898.92,5834898.99,5742200.50,0.001056,5742200.50,NaN
8,2024-04-30 13:12:00,5832898.80,5834898.99,5832898.80,0.000119,5832898.92,NaN
9,2024-04-30 13:11:00,5832898.80,5834898.80,5831898.75,0.000243,5832898.80,NaN


In [38]:
#get_account_bal test
acc = get_acc_bal()

[{'balance': '0.0', 'locked_balance': '0.0', 'currency': 'FTM'}]


In [33]:
#fetch available markets

url = "https://api.coindcx.com/exchange/v1/markets"
a = []
response = requests.get(url)
data = response.json()
for i in data:
    if i[-3:] == 'INR':
        x = i[:-3] + "_" +i[-3:]
        a.append(x)
print(str(a))

['ONDO_INR', 'AVAX_INR', 'PIXEL_INR', 'FLOKI_INR', 'PENDLE_INR', 'ZCX_INR', 'HNT_INR', 'SPELL_INR', 'XAI_INR', 'JASMY_INR', 'BCUT_INR', 'RVN_INR', 'LTC_INR', 'SAND_INR', 'TRX_INR', 'SHIB_INR', 'OOKI_INR', 'AMB_INR', 'ALPACA_INR', 'PDA_INR', 'PUSH_INR', 'XTZ_INR', 'VSC_INR', 'KSM_INR', 'DGB_INR', 'SAGA_INR', 'XDB_INR', 'STORJ_INR', 'ZBU_INR', 'GAS_INR', 'CSIX_INR', 'EOS_INR', 'LRC_INR', 'RUNE_INR', 'PORTAL_INR', 'INJ_INR', 'XVG_INR', 'SIDUS_INR', 'ARB_INR', 'ORDI_INR', 'ALGO_INR', 'BICO_INR', 'APT_INR', 'REQ_INR', 'SUSHI_INR', 'WOO_INR', 'TRB_INR', 'SUPER_INR', 'ZIL_INR', 'ARKM_INR', 'CETUS_INR', 'NFP_INR', 'FIDA_INR', 'GALA_INR', 'FIL_INR', 'LINA_INR', 'XCAD_INR', 'ALT_INR', 'RLC_INR', 'NKN_INR', 'STRK_INR', 'DIA_INR', 'EWT_INR', 'OSMO_INR', 'PYR_INR', 'LEVER_INR', 'QI_INR', 'ENS_INR', 'BTC_INR', 'WAXP_INR', 'CREAM_INR', 'CSPR_INR', 'GLM_INR', 'MEME_INR', 'SUI_INR', 'DMTR_INR', 'RSR_INR', 'LOOM_INR', 'BOME_INR', 'OCEAN_INR', 'WAVES_INR', 'AR_INR', 'WIF_INR', 'TIA_INR', 'CHR_INR', 'THET

In [34]:
#analysis and placing order 
trade_cap = 100
for i in a:
    df=data_downloader(i,"5m","50","I")
    df['rsi'] = talib.RSI(df["close"],timeperiod=15)
    rsi = df.iloc[-1]['rsi']#rsi value of the last candle
    close = df.iloc[-1]['close']#close value of last candle
    date = dt.datetime.now().date()
    time_start = dt.datetime.now()
    entry_price = df.iloc[-1]['close']
    if rsi >70:
        quantity = round(trade_cap/close,2)
        res = place_order("sell",i,df["close"],quantity)
        print(f'{i} sell trade at {close} {dt.datetime.now().time()}')
    if rsi <30:
        quantity = round(trade_cap/close,2)
        res = place_order("buy", i, df['close'],quantity)        
        print(i," ",rsi, 'buy')
        print(f'{i} buy trade at {close} {dt.datetime.now().time()}')

AMB_INR sell trade at 0.7736 13:20:25.080676
ALPACA_INR   22.876874796315448 buy
ALPACA_INR buy trade at 15.837 13:20:25.702708
PDA_INR sell trade at 7.455 13:20:26.175614
PUSH_INR sell trade at 19.825 13:20:26.704120
XTZ_INR   26.38563625446033 buy
XTZ_INR buy trade at 88.81 13:20:27.012891
VSC_INR   0.0 buy
VSC_INR buy trade at 2.1566 13:20:27.383606
KSM_INR sell trade at 2654.29 13:20:27.740127
DGB_INR   12.523284589026723 buy
DGB_INR buy trade at 1.004 13:20:28.142971
STORJ_INR sell trade at 50.0 13:20:29.284836
ZBU_INR   0.0 buy
ZBU_INR buy trade at 443 13:20:29.619453
GAS_INR   7.8956750024929985 buy
GAS_INR buy trade at 494.05 13:20:30.023307
CSIX_INR   13.397696926247468 buy
CSIX_INR buy trade at 5.2265 13:20:30.455251
EOS_INR   14.611278694634969 buy
EOS_INR buy trade at 71.08 13:20:30.847501
ORDI_INR sell trade at 3887.28 13:20:34.286007


KeyboardInterrupt: 